## Imports and Constants

In [1]:
suppressPackageStartupMessages(suppressWarnings(library(ggplot2))) # plotting
suppressPackageStartupMessages(suppressWarnings(library(dplyr))) # data manipulation
suppressPackageStartupMessages(suppressWarnings(library(patchwork))) # figure composition
suppressPackageStartupMessages(suppressWarnings(library(cowplot))) # figure composition
suppressPackageStartupMessages(suppressWarnings(library(RcppTOML))) # parsing config file
suppressPackageStartupMessages(suppressWarnings(library(lattice))) # heatmap
suppressPackageStartupMessages(suppressWarnings(library(RColorBrewer))) # heatmap
suppressPackageStartupMessages(suppressWarnings(library(ComplexHeatmap))) # heatmap
suppressPackageStartupMessages(suppressWarnings(library(ggplotify))) # grob
suppressPackageStartupMessages(suppressWarnings(library(viridis))) # color
suppressPackageStartupMessages(suppressWarnings(library(platetools))) # make plate plot
suppressPackageStartupMessages(suppressWarnings(library(circlize))) 
suppressPackageStartupMessages(suppressWarnings(library(reshape2))) # data manipulation
suppressPackageStartupMessages(suppressWarnings(library(stringr))) # string manipulation
suppressPackageStartupMessages(suppressWarnings(library(purrr))) # data manipulation
suppressPackageStartupMessages(suppressWarnings(library(VennDiagram))) # venn diagram
suppressPackageStartupMessages(suppressWarnings(library(tidyverse))) # data manipulation
suppressPackageStartupMessages(suppressWarnings(library(ggvenn))) # venn diagram
suppressPackageStartupMessages(suppressWarnings(library(grid))) # grid
suppressPackageStartupMessages(suppressWarnings(library(Polychrome)))

source("../../utils/figure_themes.r")


In [2]:
cell_type <- "PBMC"


In [3]:
# set path to anova results
anova_results_path = file.path(paste0(
    "../../../1.Exploratory_Data_Analysis/results/",cell_type,"_combined.parquet"
))
figures_path = "../figures"
if (!dir.exists(figures_path)) dir.create(figures_path)
# read in path
anova_results = arrow::read_parquet(anova_results_path)
anova_results = anova_results %>% filter(shuffled == TRUE)

In [4]:
# filter for the Cytoplasm_AreaShape_Compactness feature
anova_results %>% filter(features == "Cytoplasm_AreaShape_Compactness")

group1,group2,meandiff,lower,upper,p-adj,reject,features,p-adj_abs,pos_neg,shuffled
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<dbl>,<chr>,<lgl>
healthy,pyroptosis,0.1344,0,0.1262,0.1426,TRUE,Cytoplasm_AreaShape_Compactness,0.1426,positive,TRUE
apoptosis,pyroptosis,0.0940,0,0.0757,0.1124,TRUE,Cytoplasm_AreaShape_Compactness,0.1124,positive,TRUE
apoptosis,healthy,-0.0404,0,-0.0585,-0.0222,TRUE,Cytoplasm_AreaShape_Compactness,0.0222,negative,TRUE


In [5]:
head(anova_results)

group1,group2,meandiff,lower,upper,p-adj,reject,features,p-adj_abs,pos_neg,shuffled
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<dbl>,<chr>,<lgl>
apoptosis,healthy,-0.0084,0.2131,-0.0263,0.0095,FALSE,Nuclei_RadialDistribution_ZernikePhase_CorrPM_1_1,0.0095,positive,TRUE
apoptosis,pyroptosis,-0.0093,0.1532,-0.0274,0.0087,FALSE,Nuclei_RadialDistribution_ZernikePhase_CorrPM_1_1,0.0087,positive,TRUE
healthy,pyroptosis,-0.0009,0.9083,-0.0090,0.0071,FALSE,Nuclei_RadialDistribution_ZernikePhase_CorrPM_1_1,0.0071,positive,TRUE
apoptosis,pyroptosis,0.0039,0.7181,-0.0142,0.0220,FALSE,Cytoplasm_RadialDistribution_ZernikePhase_CorrMito_7_3,0.0220,positive,TRUE
apoptosis,healthy,0.0022,0.9021,-0.0158,0.0201,FALSE,Cytoplasm_RadialDistribution_ZernikePhase_CorrMito_7_3,0.0201,positive,TRUE
healthy,pyroptosis,0.0018,0.7182,-0.0064,0.0099,FALSE,Cytoplasm_RadialDistribution_ZernikePhase_CorrMito_7_3,0.0099,positive,TRUE


In [6]:
# create a column that adds group1 and group2 together
anova_results$group = paste0(anova_results$group1,"_",anova_results$group2)


In [7]:
# filter out rows that have p-adj_abs > 0.05
anova_results = anova_results %>% filter(`p-adj_fdr_bh` < 0.05)


ERROR: [1m[33mError[39m in `filter()`:[22m
[1m[22m[33m![39m Problem while computing `..1 = `p-adj_fdr_bh` < 0.05`.
[1mCaused by error in `mask$eval_all_filter()`:[22m
[33m![39m object 'p-adj_fdr_bh' not found


: 

In [8]:
# change the group names to replace healhty with    control
anova_results$group = str_replace_all(anova_results$group,"healthy","control")


In [9]:
# create the three df sets for a venn diagram
a_h <- anova_results %>% filter(group == "apoptosis_control")
a_p <- anova_results %>% filter(group == "apoptosis_pyroptosis")
h_p <- anova_results %>% filter(group == "control_pyroptosis")
a_h <- a_h %>% select(features)
a_p <- a_p %>% select(features)
h_p <- h_p %>% select(features)

# create a list of the three df sets
a_h_list <- a_h %>% pull(features)
a_p_list <- a_p %>% pull(features)
h_p_list <- h_p %>% pull(features)

x=list(a_h_list,a_p_list,h_p_list)


In [ ]:
# set plot size 
width <- 10
height <- 10
options(repr.plot.width=width, repr.plot.height=height)
myCol <- brewer.pal(3, "Dark2")
venn_diagram_plot <- venn.diagram(
    x=x,
    show=TRUE,
    category.names = c("Apoptosis \nvs \nControl","Apoptosis \nvs \nPyroptosis","Control \nvs \nPyroptosis"),
    filename = paste0("../figures/",cell_type,"_venn_diagram.png"),
    # Output features
    imagetype="png" , 
    resolution = 600,
    compression = "lzw",
    
    # Circles
    lwd = 2,
    lty = 'blank',
    fill = myCol,
    
    # Numbers
    cex = 1,
    fontface = "bold",
    fontfamily = "sans",
    
    # Set names
    cat.cex = 0.8,
    cat.fontface = "bold",
    cat.default.pos = "outer",
    cat.pos = c(-26, 23, 0),
    cat.dist = c(-0.03, -0.03, -0.39),
    cat.fontfamily = "sans",
    rotation = 1

)
# Set the directory path
directory <- "../figures"
# List all files in the directory
files <- list.files(directory, full.names = TRUE)
# Filter files that end with ".log"
log_files <- files[grep(".log$", files)]
# Remove the log files
file.remove(log_files)
# read in the venn diagram from PNG
venn_diagram_image_path = file.path(paste0("../figures/",cell_type,"_venn_diagram.png"))
venn_diagram_image = png::readPNG(venn_diagram_image_path)
# plot the montage image to a ggplot object
venn_diagram_image <- (
    ggplot()
    + annotation_custom(
        rasterGrob(venn_diagram_image, interpolate = TRUE),
        xmin=-Inf, xmax=Inf, ymin=-Inf, ymax=Inf
    )
    + theme_void()
)
# get the type of the object
class(venn_diagram_image)
venn_diagram_image


In [ ]:
sessionInfo()